In [1]:
import os 
from dotenv import load_dotenv, find_dotenv 
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

### **Completion Model**

In [2]:
from langchain_openai import OpenAI
llmModel = OpenAI() 

### **Chat Completion Model**

In [3]:
from langchain_openai import ChatOpenAI 
chatModel = ChatOpenAI()

#### **Prompt & Prompt templates**

In [5]:
# Completion Model 
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {curious} story about {topic}"
)

llmModelPrompt = prompt_template.format(
    curious = "fascinating",
    topic = "Humanoids" 
)

In [7]:
response = llmModel.invoke(llmModelPrompt)

In [8]:
print(response)



Once upon a time, in a distant galaxy, there was a planet called Zorion. This planet was inhabited by humanoids, a highly advanced species with remarkable abilities. They had the ability to shape-shift, control elements, and communicate telepathically.

The humanoids lived in harmony with nature and each other, using their powers for the greater good. However, their peaceful existence was threatened when a group of ruthless invaders from a neighboring planet called Xerxes attacked Zorion.

The Xerxesians were known for their greed and desire for power. They wanted to enslave the humanoids and steal their powers for themselves. The humanoids, with their advanced technology and abilities, put up a fierce fight, but were eventually outnumbered and forced to flee their planet.

The humanoids managed to escape to a nearby planet, Earth, where they sought refuge and disguised themselves as humans. They blended in seamlessly, using their abilities only when necessary. They quickly adapted t

#### **Chat Completion model**

In [11]:
# Zero Shot Prompting

from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a {profession} expert on {topic}"),
        ("human", "Hello, {profession} can you please answer a question"),
        ("ai", "Sure!"), 
        ("human", "{user_input}"),
    ] 
)

messages = chat_template.format_messages(
    profession = "Scientist", 
    topic = "Humanoids", 
    user_input = "How will humanoids evolve ?"
)

response = chatModel.invoke(messages) 
print(response.content)

As a Scientist specializing in Humanoids, I can offer some insights into how humanoids may evolve in the future. Evolution is a complex and multifaceted process that is influenced by a variety of factors, including environmental pressures, technological advancements, and societal changes.

One possible direction for the evolution of humanoids is through the integration of advanced technologies. With the rapid development of robotics, artificial intelligence, and biotechnology, it is conceivable that humanoids could become more technologically sophisticated and capable. This could involve enhancements such as increased strength, agility, intelligence, and sensory capabilities.

Additionally, as our understanding of genetics and bioengineering advances, it is possible that humanoids may undergo genetic modifications to enhance their physical and cognitive abilities. This could lead to the emergence of new humanoid species with unique characteristics designed for specific tasks or environ

In [12]:
# Few Shot Prompting 

from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {"input": "hi!", "output": "hola!"}, 
    {"input": "bye!", "output": "adios!"}
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"), 
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt, 
    examples=examples
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an English-Spanish translator"),
        few_shot_prompt, 
        ("human", "{input}") 
    ]
)

### **Chains**

In [14]:
chain = final_prompt | chatModel

response = chain.invoke({"input" : "How are you ?"})

In [15]:
print(response.content)

¿Cómo estás?


### **Output Parsers**

In [17]:
from langchain_openai import OpenAI 

llmModel = OpenAI()

In [16]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model = "gpt-3.5-turbo-0125")

In [19]:
from langchain_core.prompts import PromptTemplate 
from langchain.output_parsers.json import SimpleJsonOutputParser

json_promt = PromptTemplate.from_template(
    "Return a JSON object with an `answer` key that answers the following question: {question}"
)

json_parser = SimpleJsonOutputParser()

json_chain = json_promt | llmModel | json_parser

In [29]:
res = json_chain.invoke({"question": "What is the biggest and largest country by land and population?"})

In [30]:
res

{'answer': 'The biggest and largest country by land and population is China.'}

In [31]:
type(res)

dict

#### **Pydantic to define a custom output format**

In [37]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate 
from langchain_core.pydantic_v1 import BaseModel, Field

In [40]:
# Define a pydantic Object with the desired output format.
class Joke(BaseModel):
    setup : str = Field(description="question to set up a joke")
    punchline : str = Field(description="answer to resolve the joke")

In [41]:
# Define the parser referring the Pydantic Object
parser = JsonOutputParser(pydantic_object=Joke)

# Add the parser format instructions in the prompt defination.
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n", 
    input_variables = ["query"], 
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [43]:
# Create a chain with the prompt and the parser 
chain = prompt | chatModel | parser

chain.invoke({"query": "Tell me a Joke"})

{'setup': "Why couldn't the bicycle stand up by itself?",
 'punchline': 'Because it was two tired.'}